In [12]:
from core.data.data import Dataset
from core.data.makeedge import make_edge_files
from core.data.utils import image_transform, edge_transform
from PIL import Image
import os
from torch.utils.data import DataLoader
from torch import optim
from core.network.smoothingnetwork import  SmoothingNet
from core.loss.dataloss import DataLoss
from core.loss.smoothnessloss import SmoothnessLoss

In [4]:
ds = Dataset("images", "edges")
dl = DataLoader(ds)

In [5]:
img, edge = next(iter(dl))

In [6]:
img.shape
edge.shape

torch.Size([1, 1, 224, 224])

In [7]:
def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.Adam([input_img.requires_grad_()])
    return optimizer

In [15]:
def run(input_img, original_img, num_steps=300, device='cpu'):
    """Run the style transfer."""
    print('Building the style transfer model..')
    dataloss = DataLoss().to(device)
    smoothloss = SmoothnessLoss().to(device)
    optimizer = get_input_optimizer(input_img)
    input_img.to(device)
    original_img.to(device)

    print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            D = dataloss(original_img, input_img)
            S = smoothloss(original_img, input_img)
            
            loss = D + S
            loss.backward()

            run[0] += 1
            if run[0] % 10 == 0:
                print("run {}:".format(run))
                print('D : {:4f} S: {:4f}'.format(
                    D.item(), S.item()))
                print()

            return loss

        optimizer.step(closure)

    # a last correction...
    input_img.data.clamp_(0, 1)

    return input_img

In [ ]:
original_img = img.clone()
out = run(img, original_img)

Building the style transfer model..
Optimizing..
